In [ ]:
import pandas as pd
import torch
from sklearn.cluster import KMeans
from transformers import RobertaTokenizer, RobertaModel


# Load the pre-trained CodeBERT model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base").to('cuda:0')

def encode_code(code): #Tokenization + Input_id transform
    inputs = tokenizer(code, return_tensors="pt", truncation=True, padding=True).to('cuda:0')
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the hidden states as the vector representation
    # return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy() # features[:, 0, :] is [cls]

In [5]:
import pandas as pd
import os 
encoded_code_vectors = []
# gen_pd16 = pd.read_json("./container_data/YOUR_FINAL_FULL_AUGMENTED_DATASET_JSON_THAT_IS_ALSO_USED_FOR_TRAINING_LINEVUL_AND_CONTAINS_ORIGINAL_AND_NEW_VULS", orient="records", lines=True)
gen_pd16 = pd.read_json("./container_data/vulgen5k.jsonl", orient="records", lines=True)
gen_pd16 = gen_pd16[gen_pd16['target'] == 1]
print(gen_pd16.shape[0])

index_to_vector_map = {}

i = 0
for index, row in gen_pd16.iterrows():
    idx = str(row['idx'])
    function = str(row['func'])
    vector = encode_code(function)
    encoded_code_vectors.append(vector)
    index_to_vector_map[index] = vector
    i+=1
    # if i % 1000 == 0:
        # print(i)

encoded_code_vectors = torch.tensor(encoded_code_vectors)
print(i)

15768
15768


In [6]:
import torch
from sklearn.decomposition import PCA

vectors = encoded_code_vectors
# Step 1: Dimensionality Reduction with PCA
# Convert torch tensor to numpy for PCA
vectors_np = vectors.numpy()
pca = PCA(n_components=3)  # Reduce to 3 dimensions
reduced_vectors = pca.fit_transform(vectors_np)
reduced_vectors = torch.tensor(reduced_vectors, dtype=torch.float32)  # Convert back to torch tensor

# Step 2: Entropy Calculation with Reduced Dimensions
def calculate_entropy(vectors, num_bins):
    hist, bin_edges = torch.histogramdd(vectors, bins=num_bins)
    total_vectors = vectors.size(0)
    probabilities = hist / total_vectors
    probabilities = probabilities.flatten()
    non_zero_probabilities = probabilities[probabilities > 0]
    entropy = -torch.sum(non_zero_probabilities * torch.log(non_zero_probabilities))
    return entropy.item()

# Calculate entropy with a reasonable number of bins, e.g., 10
entropy_value = calculate_entropy(reduced_vectors, 10)

print(f"Entropy with 10 bins in reduced dimensions: {entropy_value}")

Entropy with 10 bins in reduced dimensions: 4.419580459594727
